<a href="https://colab.research.google.com/github/gautam1998/bit-predict/blob/main/notebooks/01_bit_predict_helper_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# Create a function to plot time series data
def plot_time_series(timesteps, values, format='.', start=0, end=None, label=None):
  """
  Plots a timesteps (a series of points in time) against values (a series of values across timesteps).
  
  Parameters
  ---------
  timesteps : array of timesteps
  values : array of values across time
  format : style of plot, default "."
  start : where to start the plot (setting a value will index from start of timesteps & values)
  end : where to end the plot (setting a value will index from end of timesteps & values)
  label : label to show on plot of values
  """
  # Plot the series
  plt.plot(timesteps[start:end], values[start:end], format, label=label)
  plt.xlabel("Time")
  plt.ylabel("BTC Price")
  if label:
    plt.legend(fontsize=14) # make label bigger
  plt.grid(True)

In [ ]:
def mean_absolute_scaled_error(y_true,y_pred):

  mae = tf.reduce_mean(tf.abs(y_pred - y_true))
  mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1]))

  return (mae/mae_naive_no_season)

def evaluate_preds(y_true,y_pred):

  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)
  
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) 
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true, y_pred)

  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy(),
          "mase": mase.numpy()}

In [ ]:
def get_labelled_window(data,HORIZON):

  return data[:,:-HORIZON],data[:,-HORIZON:]

def make_windows(dataset,HORIZON=1,WINDOW=7):

  window_step = tf.expand_dims(tf.range(WINDOW+HORIZON),axis=0).numpy()

  window_indices = window_step + np.expand_dims(np.arange(len(dataset) - (WINDOW+HORIZON-1)),axis=0).T

  windowed_data = dataset[window_indices]

  windows,labels = get_labelled_window(windowed_data,HORIZON)

  return windows,labels

def split_into_train_and_test(windows,labels,split_size=0.2):

  train_size = int((1-split_size)*(len(windows)))

  train_windows = windows[:train_size]
  train_labels = labels[:train_size]

  test_windows = windows[train_size:]
  test_labels = labels[train_size:]

  return train_windows,train_labels,test_windows,test_labels

In [ ]:
def build_compile_fit(results,HORIZON=1,WINDOW=7):

  full_windows,full_labels = make_windows(prices,HORIZON,WINDOW)
  print(len(full_windows),len(full_labels),len(full_windows[0]),len(full_labels[0]))

  train_windows,train_labels,test_windows,test_labels = split_into_train_and_test(full_windows,full_labels)
  print(len(train_windows),len(train_labels),len(test_windows),len(test_labels))


  tf.random.set_seed(42)

  input_layer = tf.keras.layers.Input(shape=(WINDOW,))
  dense_layer = tf.keras.layers.Dense(128,activation='relu')(input_layer)
  output_layer = tf.keras.layers.Dense(HORIZON,activation='linear')(dense_layer)

  model_name = "model_" + str(HORIZON) + "_" + str(WINDOW)
  print(model_name)
  model_temp = tf.keras.Model(input_layer,output_layer)

  model_temp.compile(
    loss = "mae",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["mae"]   
  )

  model_temp_history = model_temp.fit(train_windows,train_labels,epochs=100,verbose=0,validation_data=(test_windows,test_labels),callbacks=[SaveBestWeights("checkpoints",model_name)])

  model_temp = tf.keras.models.load_model('checkpoints/'+ model_name)
  model_temp_preds = tf.squeeze(model_temp.predict(test_windows))
  model_temp_results = evaluate_preds(tf.squeeze(test_labels),model_temp_preds)
  model_temp_results["WINDOW"] = WINDOW
  model_temp_results["HORIZON"] = HORIZON

  results = results.append(model_temp_results, ignore_index=True)

  return results
  #return model_temp_results